In [25]:
import pandas as pd 

In [26]:
df = pd.read_csv("data\gemstone.csv")
df.head()

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [27]:
df = df.drop('id',axis=1)

In [28]:
X = df.drop('price',axis=1)
y = df[['price']]

In [29]:
X

,carat,cut,color,clarity,depth,table,x,y,z
0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55
1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05
2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50
3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71
4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77
...,...,...,...,...,...,...,...,...,...
193568,0.31,Ideal,D,VVS2,61.1,56.0,4.35,4.39,2.67
193569,0.70,Premium,G,VVS2,60.3,58.0,5.75,5.77,3.47
193570,0.73,Very Good,F,SI1,63.1,57.0,5.72,5.75,3.62
193571,0.34,Very Good,D,SI1,62.9,55.0,4.45,4.49,2.81


In [30]:
y

,price
0,13619
1,13387
2,2772
3,666
4,14453
...,...
193568,1130
193569,2874
193570,3036
193571,681


In [31]:
categorical_col = X.select_dtypes(include='object').columns
numerical_col = X.select_dtypes(include='number').columns

In [32]:
numerical_col

Index(['carat', 'depth', 'table', 'x', 'y', 'z'], dtype='object')

In [33]:
categorical_col

Index(['cut', 'color', 'clarity'], dtype='object')

In [34]:
cut_categories = ['Premium', 'Very Good', 'Ideal', 'Good', 'Fair']
clarity_categories = ['VS2', 'SI2', 'VS1', 'SI1', 'IF', 'VVS2', 'VVS1', 'I1']
color_categories = ['F', 'J', 'G', 'E', 'D', 'H', 'I']

In [39]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [40]:
num_pipeline = Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='median')),
        ('scaler',StandardScaler())
    ]
)

cat_pipeline = Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='most_frequent')),
        ('encoder',OrdinalEncoder(categories=[cut_categories,color_categories,clarity_categories])),
        ('scaler',StandardScaler())
         
    ]
)

preprocessor = ColumnTransformer([
    ('num_pipeline',num_pipeline,numerical_col),
    ('cat_pipeline',cat_pipeline,categorical_col)
])

In [41]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=.30,random_state=42)

In [42]:
X_train = pd.DataFrame(preprocessor.fit_transform(X_train),columns=preprocessor.get_feature_names_out())
X_test = pd.DataFrame(preprocessor.transform(X_test),columns=preprocessor.get_feature_names_out())

In [43]:
X_train.head()

,num_pipeline__carat,num_pipeline__depth,num_pipeline__table,num_pipeline__x,num_pipeline__y,num_pipeline__z,cat_pipeline__cut,cat_pipeline__color,cat_pipeline__clarity
0,-0.823144,-1.129988,-0.641897,-0.780451,-0.835103,-0.876024,0.652424,0.058753,1.605394
1,0.945023,-1.777823,0.921902,1.073226,1.166389,0.946633,-0.381506,1.123070,-0.079246
2,1.958484,0.165682,0.400636,1.703116,1.755063,1.742237,-1.415437,1.123070,-1.202339
3,-0.995648,-0.574701,-0.641897,-1.122391,-1.161138,-1.165334,0.652424,-1.537723,2.166941
4,-0.995648,0.258230,0.400636,-1.176382,-1.152082,-1.136403,-0.381506,1.655229,0.482301


In [44]:
X_test.head()

,num_pipeline__carat,num_pipeline__depth,num_pipeline__table,num_pipeline__x,num_pipeline__y,num_pipeline__z,cat_pipeline__cut,cat_pipeline__color,cat_pipeline__clarity
0,-0.629077,0.258230,-0.120630,-0.600482,-0.581521,-0.572248,0.652424,0.590912,0.482301
1,2.605374,-2.148014,-0.120630,2.126042,2.198832,1.959219,-0.381506,-0.473406,-0.640792
2,-1.125026,-1.222536,0.921902,-1.374347,-1.414721,-1.469110,-1.415437,0.058753,2.166941
3,-1.017211,-0.574701,0.921902,-1.158385,-1.161138,-1.194265,-1.415437,1.655229,2.166941
4,0.858771,0.628421,-0.641897,0.947248,0.985258,1.004495,0.652424,1.123070,0.482301


In [45]:
from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [46]:
regressoion = LinearRegression()
regressoion.fit(X_train,y_train)

LinearRegression()

In [47]:
regressoion.coef_

array([[ 6538.82211476,  -232.4061404 ,  -177.05602534, -2090.53215704,
         -556.05883268,   -44.19321465,   -10.72128479,  -181.811724  ,
          121.17907723]])

In [48]:
regressoion.intercept_

array([3976.8787389])

In [49]:
import numpy as np
def evaluate_model(true,predicted):
    mae = mean_absolute_error(true,predicted)
    mse = mean_squared_error(true,predicted)
    rmse = np.sqrt(mean_squared_error(true,predicted))
    r2 = r2_score(true,predicted)

    return mae,rmse,r2

In [52]:
models = {
    'LinearRegression':LinearRegression(),
    'Lasso':Lasso(),
    'Ridge':Ridge(),
    'ElasticNet':ElasticNet()

}

trained_model_list = []
model_list = []
r2_list = []

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train,y_train)

    y_pred = model.predict(X_test)

    mae,rmse,r2 = evaluate_model(y_test,y_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model Train Performance')
    print('RMSE : ',rmse)
    print('MAE : ',mae)
    print('R2 score : ',r2)

    r2_list.append(r2)

    print("="*35)
    print('\n')

LinearRegression
Model Train Performance
RMSE :  1196.824881597249
MAE :  723.6034569047462
R2 score :  0.911356057138532


Lasso
Model Train Performance
RMSE :  1196.7936860863608
MAE :  725.055055982241
R2 score :  0.9113606781270963


Ridge
Model Train Performance
RMSE :  1196.8307856345216
MAE :  723.6370749375686
R2 score :  0.9113551825604018


ElasticNet
Model Train Performance
RMSE :  1632.539414426596
MAE :  1135.0835947511694
R2 score :  0.835064099195437


